# Prototype Plan: RAPTOR RAG with 2024 SEC Filings

## Overview
This notebook outlines the step-by-step plan for prototyping the RAPTOR RAG system using:
- **Data**: All 2024 SEC 10-K/10-Q filings (26,014 filings processed)
- **Models**: Test with both `gpt-oss` (13 GB) and another model via Ollama
- **Goal**: Validate complete RAPTOR pipeline before scaling to full 51 GB dataset

## Why 2024 Data Only?

**Previous approach:** 1,375 sample filings spread across 1993-2024
**New approach:** All 26K filings from 2024

**Rationale:**
- **More data = better clustering**: 26K filings vs 1,375 samples (19x more data)
- **Temporal consistency**: Same regulatory environment, accounting standards, economic conditions
- **Better testing**: Can answer "compare Apple vs Microsoft 2024 risks" type queries
- **Cleaner baseline**: Avoids format/style drift across 30+ years during prototyping
- **Statistical robustness**: More documents for RAPTOR hierarchical clustering

**Archive location:** Multi-year prototype archived in `archive_v1_multi_year/`

---

## Prototype Objectives

1. ✅ **Process all 2024 filings** - Extract, clean, chunk 26K filings (COMPLETE)
2. **Test RAPTOR hierarchical clustering** on substantial dataset
3. **Compare model performance** for summarization
4. **Verify recursive summarization** quality (3 levels)
5. **Build query interface** for retrieving and answering questions
6. **Measure performance** (speed, quality, resource usage)
7. **Identify issues** before production deployment

---

## Data Scope

**Source:** `data/external/10-X_C_2024.zip`
- **Time period:** Full year 2024 (Q1-Q4)
- **Total filings:** 26,014 (processed successfully)
- **Compressed size:** 1,611.88 MB
- **Form types:** 10-K, 10-Q, 10-K/A, 10-Q/A, 10-QT

**Actual processing output:**
- **Chunk size:** 500 tokens core + 100 token context (Anthropic method)
- **Total chunks:** 2,725,171 (2.7M)
- **Avg chunks/filing:** 104.8
- **Output file:** `processed_2024_500tok_contextual.json` (14,957 MB)
- **Processing time:** 42.1 minutes

---

## Current Status

### ✅ Step 1: Archive Multi-Year Prototype
- Moved previous work to `archive_v1_multi_year/`
- Renamed files: `01_prototype_plan_multi_year.ipynb`, `02_text_processing_multi_year.ipynb`
- Preserved all 12 chunk size outputs (200-8000 tokens)
- Key finding validated: 500-1000 tokens optimal for SEC filings

### ✅ Step 2: Text Processing (COMPLETE)
**Notebook:** `02_text_processing.ipynb`

**Status:** Successfully completed on 2025-10-16

**Results:**
1. ✅ Extracted 26,014 filings from `10-X_C_2024.zip` (100% success rate, 0 errors)
2. ✅ Parsed SRAF-XML format (metadata + clean text)
3. ✅ Chunked at **500 tokens** with **contextual chunking** (100 token context window)
4. ✅ Added contextual headers (company, CIK, form type, date)
5. ✅ Exported to `output/processed_2024_500tok_contextual.json`

**Actual output:**
- **Total chunks: 2,725,171** (2.7M chunks)
- **Avg chunks/filing: 104.8**
- **JSON file size: 14,957 MB (~15 GB)**
- **Processing time: 42.1 minutes**
- **Processing rate: 10.3 files/second**

**Token statistics:**
- Total document tokens: 1,356,067,955 (1.36 billion)
- Core tokens (stored): 1,356,067,955
- Extended tokens (embedded): 1,625,920,116 (1.63 billion)
- Context overhead: 19.9% (very efficient!)
- Avg tokens/filing: 52,128
- Min tokens/filing: 850
- Max tokens/filing: 1,071,003

**Contextual chunking configuration:**
- Core chunk: 500 tokens (stored)
- Context window: 100 tokens (50 before + 50 after)
- Extended chunk: ~700 tokens (embedded)
- Research backing: Anthropic Contextual Retrieval (35-49% improvement)

**Key improvements over original plan:**
- Used contextual chunking instead of simple overlap (Anthropic method)
- 2.7M chunks vs expected 2.9M (more efficient chunking)
- No overlap needed (context window provides continuity)
- Each chunk has both `text` (core) and `text_for_embedding` (extended)

---

## Remaining Pipeline Steps

### Step 3: Embedding Generation ⏸️
**Notebook:** `03_embedding_generation.ipynb` (CREATED)

**Tasks:**
1. Load processed chunks from Step 2
2. Load Sentence Transformers model (`all-MiniLM-L6-v2`)
3. Generate embeddings for all 2.7M chunks using `text_for_embedding` field
4. Store embeddings as NumPy array
5. Measure embedding generation time and memory usage

**Expected output:** 
- NumPy array: shape `[2,725,171, 384]`
- File size: ~4.2 GB
- Estimated time: 1-2 hours (for 2.7M chunks)

**Why all-MiniLM-L6-v2:**
- RAPTOR paper uses Sentence-BERT (same family)
- Top 20% on MTEB benchmark (56.26/100)
- 384 dimensions (compact but effective)
- ~1000 chunks/second on CPU
- 200M+ downloads, production-proven

---

### Step 4: RAPTOR Hierarchical Clustering ⏸️
**Notebook:** `04_raptor_clustering.ipynb` (to be created)

**Tasks:**
1. Load embeddings from Step 3
2. Implement global clustering (UMAP → GMM)
3. Implement local clustering within global clusters
4. Determine optimal cluster count using BIC
5. Visualize clusters (UMAP plot)
6. Validate cluster coherence (manual review)

**Expected output:**
- Cluster assignments for each chunk
- Cluster metadata (size, topic keywords)
- UMAP visualization

**Success Criteria:**
- Clusters are semantically coherent (manual review of 20+ clusters)
- No single dominant cluster (>30% of chunks)
- Reasonable cluster count (50-500 for 2.7M chunks)
- Clear topic separation in UMAP plot

**Estimated time:** 3-4 hours

---

### Step 5: Recursive Summarization (3 Levels) ⏸️
**Notebook:** `05_raptor_summarization.ipynb` (to be created)

**Tasks:**
1. Load chunks and cluster assignments
2. Test model for summarization
3. Generate **Level 1 summaries** (per-chunk)
4. Generate **Level 2 summaries** (cluster-level)
5. Generate **Level 3 summaries** (document-level)
6. Validate quality (manual review)
7. Measure summarization time

**Note:** Test on subset (1K-10K chunks) first before scaling to all 2.7M

**Estimated time:** 6-10 hours (testing on sample)

---

### Step 6: Vector Database Setup (ChromaDB) ⏸️
**Notebook:** `06_chromadb_setup.ipynb` (to be created)

**Tasks:**
1. Initialize ChromaDB
2. Create collection for 2024 SEC filings
3. Store chunks + embeddings + metadata (2.7M chunks)
4. Store summaries from chosen model
5. Test similarity search
6. Benchmark query performance

**Success Criteria:**
- All 2.7M chunks stored successfully
- Semantic search returns relevant results
- Query time < 2 seconds for top-10 retrieval
- Metadata correctly attached

**Estimated time:** 3-4 hours

---

### Step 7: RAG Query Interface ⏸️
**Notebook:** `07_rag_query_interface.ipynb` (to be created)

**Tasks:**
1. Build complete query pipeline: retrieve → augment → generate
2. Implement cluster-aware retrieval (RAPTOR)
3. Test with evaluation questions
4. Evaluate answer quality with RAGAS
5. Measure end-to-end latency
6. Compare RAPTOR vs simple RAG

**Test approach:**
- **Baseline test:** 5 questions, no RAG context
- **Simple RAG test:** 5 questions, basic retrieval
- **RAPTOR RAG test:** 5 questions, hierarchical retrieval
- **Evaluation:** RAGAS metrics (faithfulness, relevancy, precision, recall)

**Success Criteria:**
- Answers factually accurate (90%+ on manual review)
- Citations reference correct filings
- End-to-end query time < 10 seconds
- RAPTOR shows improvement over simple RAG

**Estimated time:** 3-4 hours

---

## Performance Metrics Achieved

### Step 2 Results (Text Processing):
- ✅ Filings processed: 26,014 / 26,014 (100%)
- ✅ Error rate: 0%
- ✅ Processing time: 42.1 minutes
- ✅ Rate: 10.3 files/second
- ✅ Total chunks: 2,725,171
- ✅ Output size: 14,957 MB

---

## Validation Checklist

### Before Moving to Full Dataset:

**Data Quality**
- [x] All 26K filings processed successfully (100%)
- [x] Metadata correctly parsed
- [x] Chunks maintain semantic coherence
- [x] No major data quality issues

**RAPTOR System**
- [ ] Clustering produces interpretable topics
- [ ] Level 1-3 summaries accurate and useful
- [ ] Hierarchical structure adds value
- [ ] Cluster coherence validated

**RAG Pipeline**
- [ ] ChromaDB stores/retrieves correctly
- [ ] Similarity search returns relevant chunks
- [ ] LLM generates accurate answers
- [ ] Citations work correctly

**Performance**
- [ ] Query latency acceptable (< 10 sec)
- [ ] Memory usage within limits (< 32 GB)
- [ ] No crashes or errors
- [ ] System stable under load

**Quality**
- [ ] 5+ query responses verified accurate
- [ ] Edge cases tested (long filings, unusual formats)
- [ ] RAPTOR outperforms simple RAG
- [ ] Model evaluation complete

---

## Success Definition

**Prototype is successful if:**

1. ✅ **End-to-end pipeline runs** on 26K 2024 filings (100% success rate achieved)
2. [ ] **RAPTOR clustering coherent** (manual validation of 20+ clusters)
3. [ ] **Summaries accurate** at all 3 levels (90%+ accuracy)
4. [ ] **Query responses correct** and well-cited (85%+ accuracy)
5. [ ] **Performance acceptable** (< 10 sec query, < 32 GB RAM)
6. [ ] **Measurable improvement** over simple RAG

**If successful → proceed to full dataset (51 GB) + EC2 deployment**

**If issues found → iterate on 2024 data until resolved**

---

## Resources

### Completed Files:
- ✅ `02_text_processing.ipynb` - Text extraction and chunking (DONE)
- ✅ `03_embedding_generation.ipynb` - Embedding generation notebook (CREATED)
- ✅ `output/processed_2024_500tok_contextual.json` - Processed chunks (15 GB)

### To Be Created:
- `04_raptor_clustering.ipynb`
- `05_raptor_summarization.ipynb`
- `06_chromadb_setup.ipynb`
- `07_rag_query_interface.ipynb`
- `src/models/raptor.py`

### References:
- [Anthropic Contextual Retrieval](https://www.anthropic.com/news/contextual-retrieval)
- [RAPTOR Paper](https://arxiv.org/abs/2401.18059)
- [Sentence-BERT Paper](https://arxiv.org/abs/1908.10084)
- [MTEB Benchmark](https://arxiv.org/abs/2210.07316)
- [ChromaDB Docs](https://docs.trychroma.com/)
- [Sentence Transformers](https://www.sbert.net/)
- [Ollama Python](https://github.com/ollama/ollama-python)

---

**Status:** ✅ Step 2 Complete | ⏳ Ready for Step 3 (Embedding Generation)

**Last Updated:** 2025-10-16